<a href="https://colab.research.google.com/github/FauxGrit/Titanic-Fairness/blob/main/ISYE7406_SP_Experiments_v3_price_per_bedroom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective: Initial Exploration of 'price' Model
 **Objective**: This code provides an iniital exploration for building a model to predict Airbnb prices.  In particular we are interested in learning which features are most important.

**Assumptions**: 
1. N/A

**Open Questions**:  
1. N/A

**Reference Links**
1. N/A

### Check Python Version

In [2]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

Python version
3.8.15 (default, Oct 12 2022, 19:14:39) 
[GCC 7.5.0]
Version info.
sys.version_info(major=3, minor=8, micro=15, releaselevel='final', serial=0)


### Importing Necessary Libraries

In [1]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import warnings, os, xlrd, pickle
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn import metrics
from sklearn import linear_model, preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
import statsmodels.api as sm

from pycaret.regression import *   ## <--- uncomment to run!!!
from geopy.distance import geodesic
from math import cos, sqrt

import seaborn as sns
from plotly.offline import init_notebook_mode, iplot
import plotly.figure_factory as ff
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
import plotly.graph_objs as go
import plotly
from plotly import tools

init_notebook_mode(connected=True)
pd.set_option('display.max_columns', 100)

# Supress warnings for cleaner presentation
# Turn off durng development!!!
%matplotlib inline
warnings.filterwarnings('ignore')

# Gather Data
Airbnb data from Kaggle https://www.kaggle.com/ivanovskia1/nyc-airbnb-rental-data-october-2017 .

**Preprocessing prior to data load:**
Significant data cleaning done using OpenRefine.


### Load Data into Notebook
* Load CSV file. It must be in same directory location as Jupyter Notebook (or add path to code).
* Print first five rows to confirm data load.
* Print shape to see how many rows and columns.

#### Merged and Cleaned Data

In [4]:
# Upload merged data file
df0 = pd.read_csv('Airbnb_Cleaned_Merged_v3.csv')

# Print first few rows to confirm data loaded
df0.head()

FileNotFoundError: ignored

In [ ]:
# Print shape of dataframe to determine number of rows and columns
print("Cleaned Airbnb Data\nNumber of rows: ", df0.shape[0], "\nNumber of columns: ", df0.shape[1])

Cleaned Airbnb Data
Number of rows:  39517 
Number of columns:  78


In [ ]:
df0['price_per_bdrm'] = df0['price'] / df0['bedrooms']
df0.head()

,id,host_is_superhost,Greenpoint,Chelsea,East_Harlem,Midtown,Crown_Heights,Upper_East_Side,Hells_Kitchen,Upper_West_Side,East_Village,Bushwick,Harlem,Bedford_Stuyvesant,Williamsburg,latitude,longitude,accommodates,bathrooms,bedrooms,beds,real_bed,washer_dryer,free_parking,kitchen,air_conditioning,wireless_internet,price,guests_included,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,cancellation_policy_id,reviews_per_month,Misdem,Felony,Stores,StoresArea,CensusTract,County,Borough,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Citizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,hmda_median_price,availability_30,neighbourhood_cleansed,price_per_bdrm
0,18461891,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40.774142,-73.916246,2,1.0,0,1,1,0,0,1,1,1,110,1,6,6,0,-1,4,0.00,1322,748,116,173050,36081011500,Queens,Queens,2356.0,1243.0,1113.0,18.5,59.5,0.0,0.7,16.3,1598.0,63519.0,7123.0,35294.0,4794.0,16.9,39.7,48.0,25.7,16.2,3.3,6.8,11.2,0.0,69.4,16.1,1.1,2.2,34.6,1595.0,84.7,9.1,6.2,0.0,5.5,1200000.0,0,Ditmars Steinway,inf
1,3400359,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40.866770,-73.859378,2,1.5,1,1,1,0,1,1,1,1,44,1,2,30,72,91,4,1.82,2182,984,93,175390,36005034200,Bronx,Bronx,2128.0,974.0,1154.0,37.9,8.3,47.1,0.0,6.3,1214.0,43304.0,10429.0,18132.0,2284.0,17.5,22.4,32.0,34.0,19.6,7.1,7.3,34.2,1.1,59.6,2.2,3.0,0.0,44.9,876.0,84.9,11.1,4.0,0.0,17.5,505000.0,14,Allerton,44.0
2,4407790,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40.866820,-73.858116,2,1.5,1,1,1,0,1,1,1,1,47,1,2,30,35,95,3,0.98,2074,934,87,162590,36005034400,Bronx,Bronx,2324.0,1032.0,1292.0,30.9,9.6,41.5,0.0,5.4,1385.0,67813.0,17958.0,20082.0,3207.0,14.6,14.1,21.4,29.5,24.0,13.9,11.2,40.2,16.2,41.7,0.7,0.0,1.3,45.8,1086.0,84.1,12.2,3.8,0.0,13.0,500000.0,17,Allerton,47.0
3,715270,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40.859559,-73.870669,3,1.0,1,2,1,0,0,1,1,1,39,1,2,365,131,90,4,2.17,1712,792,82,203640,36005033201,Bronx,Bronx,4125.0,1924.0,2201.0,63.5,17.7,11.3,0.0,3.1,2566.0,43038.0,6209.0,23101.0,2612.0,22.3,40.4,31.8,34.4,22.4,4.1,7.4,23.6,3.5,61.6,7.6,0.0,3.7,44.4,1898.0,80.7,14.9,4.4,0.0,14.8,255000.0,19,Allerton,39.0
4,18852048,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40.859605,-73.863609,2,1.0,1,1,1,0,1,1,1,1,49,1,3,60,1,100,1,0.24,1892,833,104,229640,36005032800,Bronx,Bronx,4260.0,2358.0,1902.0,56.0,12.2,22.6,0.0,7.1,2161.0,30089.0,6710.0,15091.0,2654.0,31.7,38.7,23.4,36.6,22.0,8.4,9.6,25.0,1.0,52.1,18.6,2.1,1.3,36.1,1567.0,75.0,18.4,6.5,0.0,15.4,560000.0,0,Allerton,49.0


# Basic Data Exploration

### Numerical Data Basic Stats

#### Clean and Merged Airbnb Data

In [ ]:
df0.describe().T

,count,mean,std,min,25%,50%,75%,max
id,39517.0,1.148467e+07,6.490729e+06,2.515000e+03,6.020237e+06,1.223286e+07,1.734794e+07,2.117803e+07
host_is_superhost,39517.0,1.103070e-01,3.132759e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
Greenpoint,39517.0,1.935876e-02,1.377842e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
Chelsea,39517.0,2.563454e-02,1.580444e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
East_Harlem,39517.0,2.705165e-02,1.622360e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
Midtown,39517.0,2.578637e-02,1.584994e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
Crown_Heights,39517.0,3.416251e-02,1.816488e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
Upper_East_Side,39517.0,3.044259e-02,1.718039e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
Hells_Kitchen,39517.0,2.826632e-02,1.657348e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
Upper_West_Side,39517.0,4.423413e-02,2.056175e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00


In [ ]:
df0.columns

Index(['id', 'host_is_superhost', 'Greenpoint', 'Chelsea', 'East_Harlem',
       'Midtown', 'Crown_Heights', 'Upper_East_Side', 'Hells_Kitchen',
       'Upper_West_Side', 'East_Village', 'Bushwick', 'Harlem',
       'Bedford_Stuyvesant', 'Williamsburg', 'latitude', 'longitude',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'real_bed',
       'washer_dryer', 'free_parking', 'kitchen', 'air_conditioning',
       'wireless_internet', 'price', 'guests_included', 'minimum_nights',
       'maximum_nights', 'number_of_reviews', 'review_scores_rating',
       'cancellation_policy_id', 'reviews_per_month', 'Misdem', 'Felony',
       'Stores', 'StoresArea', 'CensusTract', 'County', 'Borough', 'TotalPop',
       'Men', 'Women', 'Hispanic', 'White', 'Black', 'Native', 'Asian',
       'Citizen', 'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr',
       'Poverty', 'ChildPoverty', 'Professional', 'Service', 'Office',
       'Construction', 'Production', 'Drive', 'Carpool', 'Transit',

# 200 Experiment with 2.2 Auto-ML

References:  https://towardsdatascience.com/creating-the-whole-machine-learning-pipeline-with-pycaret-db39a3006840  https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Intermediate%20-%20REG102.ipynb and https://www.kdnuggets.com/2020/11/5-things-doing-wrong-pycaret.html

This section is based on the PyCaret tutorial for regession found at https://www.pycaret.org/tutorials/html/REG101.html. From this article: 
> PyCaret's Regression module (pycaret.regression) is a supervised machine learning module which is used for predicting continuous values / outcomes using various techniques and algorithms. Regression can be used for predicting values / outcomes such as sales, units sold, temperature or any number which is continuous. 

> PyCaret's regression module has over 25 algorithms and 10 plots to analyze the performance of models. Be it hyper-parameter tuning, ensembling or advanced techniques like stacking, PyCaret's regression module has it all.

### 200.6 Setting up PyCaret Environment
> The setup() function initializes the environment in pycaret and creates the transformation pipeline to prepare the data for modeling and deployment. setup() must be called before executing any other function in pycaret. It takes two mandatory parameters: a pandas dataframe and the name of the target column. All other parameters are optional and are used to customize the pre-processing pipeline (we will see them in later tutorials).

> When setup() is executed, PyCaret's inference algorithm will automatically infer the data types for all features based on certain properties. The data type should be inferred correctly but this is not always the case. To account for this, PyCaret displays a table containing the features and their inferred data types after setup() is executed. If all of the data types are correctly identified enter can be pressed to continue or quit can be typed to end the expriment. Ensuring that the data types are correct is of fundamental importance in PyCaret as it automatically performs a few pre-processing tasks which are imperative to any machine learning experiment. These tasks are performed differently for each data type which means it is very important for them to be correctly configured.

Note that we use the numeric_features() to force the classification of features. 
- Iterative imputation has a default of 5
- Light Gradiant Boosting Machine is chosen as the imputation method
- The data is normalized using the 'robust' setting
- Robust normalization scales according to the interquartile ranges and is recommended for data with outliers

Documentation on setup():  https://pycaret.org/regression/

In [ ]:
# Note we are forcing the classification of all features that are not correctly implied by setup()
# Ignore 'id' since it is a key (otherwise makes many categorical vars)
exp_reg101 = setup(data = df0, target = 'price_per_bdrm',  log_experiment = False, experiment_name = '1 initial experiment', \
                   ignore_features = ['price','id','CensusTract','County','Borough','neighbourhood_cleansed'], train_size = 0.9,\
                   imputation_type = 'iterative', numeric_iterative_imputer = 'lightgbm',\
                   normalize=True, normalize_method='robust', pca=False, numeric_features = \
                   ['availability_30','host_is_superhost','Greenpoint', 'Chelsea', 'East_Harlem',\
                    'Midtown', 'Crown_Heights', 'Upper_East_Side', 'Hells_Kitchen',\
                    'Upper_West_Side', 'East_Village', 'Bushwick', 'Harlem',\
                    'Bedford_Stuyvesant', 'Williamsburg', 'latitude', 'longitude',\
                    'accommodates', 'bathrooms', 'bedrooms', 'beds', 'real_bed',\
                    'washer_dryer', 'free_parking', 'kitchen', 'air_conditioning',\
                    'wireless_internet', 'guests_included', 'minimum_nights',\
                    'maximum_nights', 'number_of_reviews', 'review_scores_rating',\
                    'cancellation_policy_id', 'reviews_per_month', 'Misdem', 'Felony',\
                    'Stores', 'StoresArea', 'TotalPop','hmda_median_price',\
                    'Men', 'Women', 'Hispanic', 'White', 'Black', 'Native', 'Asian',\
                    'Citizen', 'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr',\
                    'Poverty', 'ChildPoverty', 'Professional', 'Service', 'Office',\
                    'Construction', 'Production', 'Drive', 'Carpool', 'Transit', 'Walk',\
                    'OtherTransp', 'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork',\
                    'PublicWork', 'SelfEmployed', 'FamilyWork', 'Unemployment'])


,Description,Value
0,session_id,3900
1,Target,price_per_bdrm
2,Original Data,"(39517, 79)"
3,Missing Values,True
4,Numeric Features,72
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(32347, 72)"


> Once the setup has been succesfully executed it prints the information grid which contains several important pieces of information. Most of the information is related to the pre-processing pipeline which is constructed when setup() is executed. The majority of these features are out of scope for the purposes of this tutorial however a few important things to note at this stage include:

* session_id : A pseduo-random number distributed as a seed in all functions for later reproducibility. If no session_id is passed, a random number is automatically generated that is distributed to all functions. In this experiment, the session_id is set as 23 for later reproducibility.

* Original Data : Displays the original shape of dataset. In this experiment (23, 29) means 23 samples and 29 features including the target column.  The samples include client.matter used as a key vs. a predictive feature.

* Missing Values : When there are missing values in the original data this will show as True. For this experiment there are several missing values in the dataset.  PyCaret auto imputed values using mean.  This is configurable.

* Numeric Features : Number of features inferred as numeric. Note we forced several in the setup() function above

* Categorical Features : Number of features inferred as categorical. 

* Transformed Train Set : Displays the shape of the transformed training set. Notice that the original shape of (23, 29) is transformed into (16, 43) for the transformed train set. The number of features has increased from 29 from 43 due to categorical encoding

* Transformed Test Set : Displays the shape of transformed test/hold-out set. There are 1621 samples in test/hold-out set. This split is based on the default value of 70/30 that can be changed using train_size parameter in setup.

> Notice how a few tasks that are imperative to perform modeling are automatically handled such as missing value imputation, categorical encoding etc. Most of the parameters in setup() are optional and used for customizing the pre-processing pipeline. 

In [ ]:
# Call help function to see what data is in the resulting object
help(get_config)

Help on function get_config in module pycaret.regression:

get_config(variable: str)
    This function retrieves the global variables created when initializing the 
    ``setup`` function. Following variables are accessible:
    
    - X: Transformed dataset (X)
    - y: Transformed dataset (y)  
    - X_train: Transformed train dataset (X)
    - X_test: Transformed test/holdout dataset (X)
    - y_train: Transformed train dataset (y)
    - y_test: Transformed test/holdout dataset (y)
    - seed: random state set through session_id
    - prep_pipe: Transformation pipeline
    - fold_shuffle_param: shuffle parameter used in Kfolds
    - n_jobs_param: n_jobs parameter used in model training
    - html_param: html_param configured through setup
    - create_model_container: results grid storage container
    - master_model_container: model storage container
    - display_container: results display container
    - exp_name_log: Name of experiment
    - logging_param: log_experiment param
 

In [ ]:
# Print data following transformation.  Note imputation of missing values.
# investigate TrgtMultiNat_0.0, TrgtMultiNat_1.0, RegHurdle_not_available

trainingData = get_config('X_train')

# testX.head().T
trainingData.describe().T

,count,mean,std,min,25%,50%,75%,max
host_is_superhost,32347.0,0.112159,0.315623,0.000000,0.000000,0.0,0.000000,1.000000
Greenpoint,32347.0,0.020466,0.141602,0.000000,0.000000,0.0,0.000000,1.000000
Chelsea,32347.0,0.022722,0.148998,0.000000,0.000000,0.0,0.000000,1.000000
East_Harlem,32347.0,0.028194,0.165525,0.000000,0.000000,0.0,0.000000,1.000000
Midtown,32347.0,0.021177,0.143964,0.000000,0.000000,0.0,0.000000,1.000000
...,...,...,...,...,...,...,...,...
SelfEmployed,32347.0,0.007470,0.693928,-1.551020,-0.489796,0.0,0.510204,2.775510
FamilyWork,32347.0,0.104779,0.320889,0.000000,0.000000,0.0,0.000000,3.900000
Unemployment,32347.0,0.156029,0.715016,-1.149254,-0.402985,0.0,0.597015,4.253731
hmda_median_price,32347.0,0.192397,0.996798,-1.716981,-0.452830,0.0,0.547170,10.547170


Compare Models
> Comparing all models to evaluate performance is the recommended starting point for modeling once the setup is completed (unless you exactly know what kind of model you need, which is often not the case). This function trains all models in the model library and scores them using kfold cross validation for metric evaluation. The output prints a score grid that shows average MAE, MSE, RMSE, R2, RMSLE and MAPE accross the folds (10 by default) of all the available models in the model library.

We are forcing the model to sort by the r-squared value, to round off to 2 digits, and limiting the folds given the extremely small data set. 

Documentation for parameters found:  https://pycaret.readthedocs.io/en/latest/api/regression.html

In [ ]:
compare_models(sort="MSE", round=2, fold=10)

IntProgress(value=0, description='Processing: ', max=99)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:20:35
Status,. . . . . . . . . . . . . . . . . .,Compiling Final Models
Estimator,. . . . . . . . . . . . . . . . . .,Huber Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,0.00,0.00,0.00,0.00,0.00,0.00,1.15
lightgbm,Light Gradient Boosting Machine,42.90,24353.70,144.99,0.14,0.48,0.42,0.42
br,Bayesian Ridge,44.92,24442.00,144.84,0.15,0.52,0.46,0.10
ridge,Ridge Regression,45.09,24442.14,144.86,0.15,0.53,0.46,0.04
lr,Linear Regression,45.20,24446.92,144.89,0.15,0.53,0.46,0.38
lasso,Lasso Regression,44.76,24560.63,145.29,0.15,0.51,0.45,0.22
omp,Orthogonal Matching Pursuit,45.55,24680.71,145.71,0.14,0.52,0.46,0.02
catboost,CatBoost Regressor,40.97,24842.14,145.68,0.14,0.46,0.40,4.13
en,Elastic Net,46.62,24923.86,146.72,0.13,0.52,0.48,0.06
gbr,Gradient Boosting Regressor,42.40,25712.23,148.40,0.11,0.47,0.42,2.43


AttributeError: 'str' object has no attribute 'decode'

From reference material regarding "compare models()":
> Two simple words of code (not even a line) have created over 22 models using 10 fold cross validation and evaluated the 6 most commonly use regression metrics (MAE, MSE, RMSE, R2, RMSLE and MAPE). The score grid printed above highlights the highest performing metric for comparison purposes only. The grid by default is sorted using R2 (highest to lowest) which can be changed by passing sort parameter. For example compare_models(sort = 'RMSLE') will sort the grid by RMSLE (lower to higher since lower is better). If you want to change the fold parameter from the default value of 10 to a different value then you can use the fold parameter. For example compare_models(fold = 5) will compare all models on 5 fold cross validation. Reducing the number of folds will improve the training time.

### 200.8 Create Models based on Comparison above

From the reference material:
> While compare_models() is a powerful function and often a starting point in any experiment, it does not return any trained models. PyCaret's recommended experiment workflow is to use compare_models() right after setup to evaluate top performing models and finalize a few candidates for continued experimentation. As such, the function that actually allows to you create a model is unimaginatively called create_model(). This function creates a model and scores it using stratified cross validation. Similar to compare_models(), the output prints a score grid that shows MAE, MSE, RMSE, R2, RMSLE and MAPE by fold.

Below we will explore the top three models from the 25 available:

* Elastic Net 'ada'
* CatBoost Regressor 'catboost'
* Linear Regression 'lr'

Doc reference:  https://pycaret.org/create-model/

### Create Huber Regression Model

In [ ]:
huberReg = create_model('huber', round = 3)

In [ ]:
# tuned_huberReg = tune_model(huberReg, round=3)
# tuned_huberReg

In [ ]:
plot_model(huberReg, plot='feature')

### 200.8.1 Create Light Gradient Boosting Machine Model

In [ ]:
lightgbm = create_model('lightgbm', round = 3)

In [ ]:
tuned_lightgbm = tune_model(lightgbm, round=3)
tuned_lightgbm

In [ ]:
plot_model(tuned_lightgbm, plot='feature')

In [ ]:
evaluate_model(tuned_lightgbm)

### Create Linear Regression Model

In [ ]:
linear_reg = create_model('lr', round = 3)

In [ ]:
plot_model(linear_reg, plot='feature')

### Explore Recursive Feature Elimination for Linear Regression
This section uses the sklearn library in Python.

In [ ]:
X_train_pc = get_config('X_train')
y_train_pc = get_config('y_train')

X_test_pc = get_config('X_test')
y_test_pc = get_config('y_test')

In [ ]:
# Running RFE with the output number of the variable equal to 10
lm = LinearRegression()
rfe = RFE(lm, n_features_to_select=5)             # running RFE
rfe = rfe.fit(X_train_pc, y_train_pc)
print(rfe.support_)           # Printing the boolean results
print(rfe.ranking_) 

In [ ]:
X_train_pc.loc[:, rfe.support_].columns

### Linear Regression Iteration 1
Create a linear regression model using all the variables. Then iterate buy creating a new model that only uses the variables with a p-value of 0.000. This is a quick method for identifying features with higher importance to the linear regression model.

In [ ]:
lr_model = lm.fit(X_train_pc, y_train_pc)
lr_model

In [ ]:
model = sm.OLS(y_train_pc, X_train_pc).fit()
predictions = model.predict(X_train_pc) 
print_model = model.summary()
print(print_model)

### Linear Regression Iteration 2

In [ ]:
# Reduce to onlyy those with p-value = .000
X_train_1 = X_train_pc[['host_is_superhost',\
             'accommodates', 'bathrooms', 'bedrooms','availability_30',\
             'washer_dryer', 'kitchen', \
             'wireless_internet', \
             'review_scores_rating',\
             'reviews_per_month', 'Misdem', 'Felony',\
             'Stores', 'TotalPop',\
             'Men', 'Women', \
             'IncomePerCap','hmda_median_price',\
             'Drive', 'Carpool', 'Transit', 'Walk',\
             'OtherTransp', 'WorkAtHome', 'MeanCommute', 'Employed']]

In [ ]:
model2 = sm.OLS(y_train_pc, X_train_1).fit()
predictions = model2.predict(X_train_1) 

print_model = model2.summary()
print(print_model)

### Linear Regression Iteration 2

In [ ]:
# Reduce to onlyy those with p-value = .000
X_train_2 = X_train_pc[['host_is_superhost',\
             'accommodates', 'bathrooms', 'bedrooms',\
             'washer_dryer', 'kitchen', \
             'wireless_internet', \
             'review_scores_rating',\
             'reviews_per_month', 'Felony',\
             'TotalPop',\
             'Men', 'Women', \
             'IncomePerCap',\
             'Transit', 'Walk','hmda_median_price',\
             'OtherTransp','MeanCommute', 'availability_30']]

In [ ]:
model2 = sm.OLS(y_train_pc, X_train_2).fit()
predictions = model2.predict(X_train_2) 
model2.summary()